# Simple prospective simulation

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, @guerrero_oa)

In this tutorial I will demonstrate how to perform a simple prospective analysis by simulating the indicators forward 10 years and measuring the development gaps that would remain open. I will also show how to implement some intuitive visualisations.

## Importing Python's libraries to manipulate and visualise data

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing PPI's functions

In [3]:
import requests
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

## Load data

Since we are now performing prospective simulations, the initial conditions are the most updated values, i.e. `IF`.

In [17]:
# INDICATORS
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')
N = len(df_indis)
I0 = df_indis.IF.values # initial values
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix
Imax = df_indis.maxVals.values.astype(float)
Imin = df_indis.minVals.values

# PARAMETERS
df_params = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/parameters.csv')
alphas = df_params.alpha.values
alphas_prime = df_params.alpha_prime
betas = df_params.beta

# NETWORK
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')
A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

# DISBURSEMENT SCHEDULE
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')
Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

# RELATIONAL TABLE
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')
B_dict = {}
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Simulate

In [18]:
T = 30

output = ppi.run_ppi(I0, alphas, alphas_prime, betas, A=A, R=R, qm=qm, rl=rl,
            Imax=Imax, Imin=Imin, Bs=Bs, B_dict=B_dict, T=T)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
df_indis

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,minVals,maxVals,instrumental,seriesName,color,I0,IF,successRates,qm,rl
0,0.092638,0.092244,0.094173,0.096063,0.097323,0.099016,0.099961,0.101772,0.103504,0.105315,...,0.0,1.0,0,"Unemployment rate (% of total labor force, age...",#A21942,0.092638,0.130472,0.909091,0.5,0.5
1,0.587879,0.576970,0.566061,0.550303,0.534545,0.507879,0.480000,0.450909,0.420606,0.387879,...,0.0,1.0,1,Demand for family planning satisfied by modern...,#FF3A21,0.587879,0.204848,0.050000,0.5,0.5
2,0.000344,0.000344,0.000344,0.000344,0.000344,0.000344,0.000344,0.000344,0.000344,0.000344,...,0.0,1.0,1,Proportion of urban population living in slums...,#FD9D24,0.000344,0.000022,0.050000,0.5,0.5
3,0.016777,0.016873,0.016956,0.017025,0.017094,0.017163,0.017231,0.017300,0.017369,0.017438,...,0.0,1.0,1,Poverty headcount ratio at $1.90/day (%),#E5243B,0.016777,0.004766,0.681818,0.5,0.5
4,0.645282,0.647417,0.649903,0.652505,0.654835,0.656621,0.657806,0.658524,0.659029,0.659650,...,0.0,1.0,1,Poverty headcount ratio at $3.20/day (%),#E5243B,0.645282,0.549709,0.545455,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.381386,0.380243,0.379100,0.377957,0.376843,0.375771,0.374743,0.373743,0.372786,0.371814,...,0.0,1.0,1,Press Freedom Index (best 0-100 worst),#00689D,0.381386,0.376286,0.181818,0.5,0.5
68,0.323077,0.323077,0.323077,0.323077,0.323077,0.323077,0.323077,0.323077,0.321538,0.321538,...,0.0,1.0,1,Access to and affordability of justice (worst ...,#00689D,0.323077,0.349231,0.227273,0.5,0.5
69,0.399800,0.401333,0.416600,0.413200,0.412067,0.416867,0.421200,0.423333,0.407467,0.410333,...,0.0,1.0,1,Government spending on health and education (%...,#19486A,0.399800,0.459400,0.727273,0.5,0.5
70,0.348867,0.332767,0.360167,0.361767,0.351633,0.337167,0.328333,0.308700,0.318933,0.335233,...,0.0,1.0,1,Other countries: Government revenue excluding ...,#19486A,0.348867,0.647900,0.590909,0.5,0.5
